In [ ]:
!pip install openai python-dotenv

## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

In [17]:
import os
from openai import OpenAI
import openai

from dotenv import load_dotenv, find_dotenv

In [18]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [19]:
llm_model = "gpt-4"

In [20]:
prompt = 'Hello, I\'m using api to access the llm. Tell me more about you'

In [21]:
client = OpenAI()

stream = client.chat.completions.create(
    model=llm_model,
    messages = [{'role': 'user', 'content': prompt}]
)

In [22]:
stream

ChatCompletion(id='chatcmpl-9iF1WuWcgic2K0WEaXe7IWSYxaJHK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="As an AI developed by OpenAI, I don't have personal experiences or attributes. However, I am designed to provide information, support, and answer questions to the best of my ability. I use machine learning models to understand and generate text based on the input I receive. I'm here to help with a wide range of requests, from general information to specific tasks. Is there anything in particular you would like to know?", role='assistant', function_call=None, tool_calls=None))], created=1720331210, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=85, prompt_tokens=24, total_tokens=109))

In [42]:
## Chai API: OpenAI
def get_completion(prompt, model=llm_model):
    messages = [{'role': 'user', 'content': prompt}]
    response = client.chat.completions.create(
        model=llm_model,
        messages = [{'role': 'user', 'content': prompt}],
        temperature=0,
    )
    return response.choices[0].message.content

In [45]:
get_completion('Hello, I\'m using api to access the llm. Tell me more about you')

"As an AI developed by OpenAI, I don't have personal experiences or emotions, but I'm designed to understand and generate human-like text based on the input I receive. I can assist with a wide range of tasks, from answering questions and providing explanations to creating written content and more. I'm constantly learning and improving from a vast amount of information, but I don't store personal data unless explicitly provided for the purpose of the conversation. I'm here to help you, so feel free to ask me anything!"

In [50]:
customer_email = """Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!"""

style = """American English in a calm and respectful tone"""

prompt = f"""Translate the text thta is delimited by triple backticks into a stype that is {style}. text: ```{customer_email}```"""

In [51]:
print(prompt)

Translate the text thta is delimited by triple backticks into a stype that is American English in a calm and respectful tone. text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!```


In [52]:
response = get_completion(prompt)
response

"I'm quite upset that the lid of my blender flew off and splattered my kitchen walls with smoothie. To add to the frustration, the warranty doesn't cover the cost of cleaning up my kitchen. I need your assistance immediately, please."

### Chat API: LangChain

In [ ]:
!pip install --upgrade langchain

In [ ]:
!pip install langchain-openai

#### Model

In [62]:
from langchain_openai import ChatOpenAI

In [63]:
chat = ChatOpenAI(temperature=0.0, model=llm_model) 

#### Prompt Template

In [64]:
template_string = """Translate the text that is delimited by tripple backticks into a style that is {style}.\text: ```{text}```""" 

In [72]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [73]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by tripple backticks into a style that is {style}.\text: ```{text}```')

In [75]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [77]:
customer_messages = prompt_template.format_messages(
    style=style,
    text=customer_email)

In [78]:
customer_messages

[HumanMessage(content="Translate the text that is delimited by tripple backticks into a style that is American English in a calm and respectful tone.\text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!```")]

In [79]:
customer_response = chat(customer_messages)

/home/agastya/mambaforge/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [82]:
customer_response

AIMessage(content="I'm quite upset that my blender lid came off and splattered my kitchen walls with smoothie. To add to the frustration, the warranty doesn't cover the cost of cleaning up my kitchen. I need your assistance immediately, please.", response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 89, 'total_tokens': 136}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2ac29af5-3268-4d17-b5ba-490525e01577-0', usage_metadata={'input_tokens': 89, 'output_tokens': 47, 'total_tokens': 136})

In [81]:
customer_response.content

"I'm quite upset that my blender lid came off and splattered my kitchen walls with smoothie. To add to the frustration, the warranty doesn't cover the cost of cleaning up my kitchen. I need your assistance immediately, please."

### One more example

In [83]:
style_2 = """a polite and funny tone that speaks in English pirate"""

In [84]:
service_message = prompt_template.format_messages(style=style_2, text=customer_email)
service_message

[HumanMessage(content="Translate the text that is delimited by tripple backticks into a style that is a polite and funny tone that speaks in English pirate.\text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!```")]

In [85]:
service_response = chat(service_message)
service_response

AIMessage(content="Blimey, I be spittin' feathers that me blender lid took flight and splattered me galley walls with fruit grog! And to add insult to injury, the warranty don't be coverin' the cost of swabbin' the deck of me galley. I be in dire need of yer assistance this very moment, shipmate!", response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 91, 'total_tokens': 164}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0a813df8-ff38-4942-bede-94cb6c4bc0f3-0', usage_metadata={'input_tokens': 91, 'output_tokens': 73, 'total_tokens': 164})

## Output Parser
Defining the output structure of the LLM Output

In [87]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [88]:
customer_review = """This leaf blower is pretty amazing. It has four settings: candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."""

review_template = """
For the following text, extract the following information
gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days dd it take for the product to arrive? If this informattion is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [89]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)

In [90]:
prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='\nFor the following text, extract the following information\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days dd it take for the product to arrive? If this informattion is not found, output -1.\n\nprice_value: Extract any sentences about the value or price, and output them as a comma separated Python list.\n\nformat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))])

In [92]:
messages=prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model = llm_model)
response = chat(messages)

In [95]:
response.content

'{\n"gift": true,\n"delivery_days": 2,\n"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}'

In [97]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

Since the llm output is pure string & we need the llm output to be formatted in json

#### Parsing the LLM output string into a Python Dictionary

In [98]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [100]:
gift_schema = ResponseSchema(name='gift',
                             description="Was the item purchased as a gift forr someone else? \
                             Answer True if yes, False if not or unknown")
delivery_days_schema = ResponseSchema(name='delivery_days',
                                     description="How many days dd it take for the product to arrive? \
                                     If this informattion is not found, output -1.")
price_value_schema = ResponseSchema(name='price_value',
                                    description="Extract any sentences about the value or price, \
                                    and output them as a comma separated Python list.")
response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [101]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [102]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased as a gift forr someone else?                              Answer True if yes, False if not or unknown\n\t"delivery_days": string  // How many days dd it take for the product to arrive?                                      If this informattion is not found, output -1.\n\t"price_value": string  // Extract any sentences about the value or price,                                     and output them as a comma separated Python list.\n}\n```'

In [103]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [104]:
messages[0].content

'For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing. It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "``

In [106]:
response = chat(messages)
response

AIMessage(content='```json\n{\n\t"gift": "True",\n\t"delivery_days": "2",\n\t"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```', response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 326, 'total_tokens': 379}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-68fad490-e0cf-4de7-b573-ff2a7a1bf1eb-0', usage_metadata={'input_tokens': 326, 'output_tokens': 53, 'total_tokens': 379})

In [107]:
response.content

'```json\n{\n\t"gift": "True",\n\t"delivery_days": "2",\n\t"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```'

In [110]:
output_dict = output_parser.parse(response.content)

In [111]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [112]:
output_dict['gift']

'True'